In [14]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from tqdm import tqdm
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textstat import flesch_reading_ease,flesch_kincaid_grade
import copy
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV


In [37]:
class Features:
    def __init__(self, df_tr, df_te):
        self.vectorizer = CountVectorizer(ngram_range=(1,3), min_df=5)
        self.vectorizer_TFIDF  = TfidfVectorizer(ngram_range=(1,3),min_df=5,norm='l2')
        
        all_tweets = df_tr.tweet
        all_tweets=all_tweets.append(df_te.tweet)

        # fit was used to learn the vocabulary
        
        TempVector = self.vectorizer.fit(all_tweets)
        TempVectorTfidf = self.vectorizer_TFIDF.fit(all_tweets)
        
        # Getting the vocabulary for ngrams
        self.Vocab_ngrams = TempVector.get_feature_names()
        self.Vocab_ngrams_Tfidf = TempVectorTfidf.get_feature_names()
        
        self.TrainTags=[]
        self.TestTags=[]
        for i in (range(len(df_tr.tweet))):
            tags=nltk.pos_tag(df_tr.tweet[i].split(" "))
            tag_tweet=[]
            for j in tags:
                tag_tweet.append(j[1])
            self.TrainTags.append( " ".join(tag_tweet))

        for i in tqdm(range(len(df_te.tweet))):
            tags=nltk.pos_tag(df_te.tweet[i].split(" "))
            tag_tweet=[]
            for j in tags:
                tag_tweet.append(j[1])
            self.TestTags.append( " ".join(tag_tweet))

        print("ok")
        self.vectorizer_pos = CountVectorizer(ngram_range=(1,3),min_df=5)
        Alltags=copy.deepcopy(self.TrainTags)
        Alltags.extend(self.TestTags)
        self.vectorizer_pos.fit(Alltags)
        self.pos_vocab=self.vectorizer_pos.get_feature_names()
        print("ok2")
        
    def FuncAllCapsCount(self, GivenTweets):
        # Function to get All caps count from the tweets
        # pass tweets as it is
        
        AllCapsCount = np.zeros(len(GivenTweets))
        for i in tqdm(range(len(GivenTweets))):
            tweet = nltk.word_tokenize(GivenTweets[i])
            for word in tweet:
                if(word!="I" and word != "USER" and word != "URL" and re.match("^[A-Z]+$",word)):
                    AllCapsCount[i] += 1
                    
        return AllCapsCount
    
    def FuncPunctuationsCount(self, GivenTweets):
        # Function to get COntinuous Punctuation (???,??!,!!) Count and check if last token was a ? or !
        # pass tweets as it is
        
        ContPunctuationCount=np.zeros(len(GivenTweets))
        LastTokenPunctuation=np.zeros(len(GivenTweets))
        
        for i in tqdm(range(len(GivenTweets))):
            tweet = nltk.word_tokenize(GivenTweets[i])
            token = 0

            while token < len(tweet):
                if(tweet[token]=="?" or tweet[token]=="!"):
                    index=token+1
                    while(index< len(tweet) and (tweet[index]=="?" or tweet[index]=="!")):
                        index+=1
                    if(index-token>1):
                        ContPunctuationCount[i]+=1
                        token=index
                        
                token += 1
                
            if tweet[len(tweet)-1] == "?" or tweet[len(tweet)-1] == "!":
                LastTokenPunctuation[i] += 1

        return ContPunctuationCount, LastTokenPunctuation
    
    def FuncHashtagsCount(self, GivenTweets):    
        # Fucntion to get HashTag count from the tweets
        # pass tweets as it is
        
        HashTagsCount=np.zeros(len(GivenTweets))
        for i in tqdm(range(len(GivenTweets))):
            tweet=GivenTweets[i].split(" ")
            for word in tweet:
                if(re.match("^#[a-zA-Z][a-zA-Z0-9]*",word)):
                    HashTagsCount[i]+=1
        return(HashTagsCount)    
    
    def Func_NRC_Hashtag_Emotion(self, GivenTweets):
        # NRC-Hashtag-Emotion-Lexicon
        
        d_emotion=pd.read_csv("./lexicons/5. NRC-Hashtag-Emotion-Lexicon-v0.2.txt", sep='\t', names=['emotion','word','score'], header=None)
        dic_dEmo={}
        for i in range(len(d_emotion['word'])):
            if d_emotion['emotion'][i]=="anger":
                dic_dEmo[d_emotion['word'][i]]=d_emotion['score'][i]
        aggEmoHashtags=np.zeros(len(GivenTweets))
        for i in range(len(GivenTweets)):
            tweet=GivenTweets[i]
            hashtags = [j  for j in tweet.split() if j.startswith("#") ]
            for tag in hashtags:
                word=tag.lower()
                if word in dic_dEmo:
                    aggEmoHashtags[i]+=dic_dEmo[word]
                elif word[1:] in dic_dEmo:
                    aggEmoHashtags[i]+=dic_dEmo[word[1:]]
        return(aggEmoHashtags)

        #     Vector aggEmoHashtags conatains Aggregate emotion score (Hashtags)
        #     Adding features into dataframe
        
    def Func_NRC_10_Expanded_Emotion(self, GivenTweets):
        # NRC-Hashtag-Emotion-Lexicon
        
        d_emotion=pd.read_csv("./lexicons/6. NRC-10-expanded.csv", sep='\t')
        dic_dEmo={}
        for i in range(len(d_emotion['word'])):
            dic_dEmo[d_emotion['word'][i]]=d_emotion['anger'][i]
        aggScoreEmo=np.zeros(len(GivenTweets))
        for i in range(len(GivenTweets)):
    #         print(i)
            tweet=GivenTweets[i]
            for word in word_tokenize(tweet):
                if word in dic_dEmo:
    #                 print(word)
                    aggScoreEmo[i]+=dic_dEmo[word]
        return(aggScoreEmo)      

    def Func_NRC_word_emotion_lexicon(self, GivenTweets):
        emotion_count=np.zeros(len(GivenTweets))
        emotions_df=pd.read_csv("./lexicons/8. NRC-word-emotion-lexicon.txt", sep='\t', names=['word','emotion','score'], header=None)
        # This set contains words with Emotion X=1 in above dataframe
        set_emo=set()
        for i in range(len(emotions_df['word'])):
            if emotions_df['emotion'][i]=="anger" and emotions_df['score'][i]==1:
                set_emo.add(emotions_df['word'][i])

        for i in range(len(GivenTweets)):
            tweet=word_tokenize(GivenTweets[i])
            for word in tweet:
                word_lower=word.lower()
                if word_lower in set_emo:
                    emotion_count[i]+=1
        return(emotion_count)    
    
    def func_emoji_feature(self, training,testing):
        emoji_vocab = set()
        
        for i in training:
            temp_lis=i.split(" ")
            for word in temp_lis:
                for character in word:
                    if(character!=emoji.demojize(character)):
                        for j in character:
                            emoji_vocab.add(j)
        for i in testing:
            temp_lis=i.split(" ")
            for word in temp_lis:
                for character in word:
                    if(character!=emoji.demojize(character)):
                        for j in character:
                            emoji_vocab.add(j)
        print(len(emoji_vocab))
        
    def n_grams(self, tweets,flag="tfidf"):

            if(flag=="tfidf"):
                ngrams_ft = self.vectorizer_TFIDF.transform(tweets)
            else:
                ngrams_ft = self.vectorizer.transform(tweets)


    #         te_ngrams = vectorizer.transform(df_testing["tweet"])        
            return ngrams_ft
    
    def slang(self, GivenTweets):
        slangs= pd.read_csv("./lexicons/SlangSD/SlangSD.txt", sep='\t', names=['word','score'], header=None)
        slangScore=np.zeros(len(GivenTweets))
        dic_slangs={}
        for i in (range(len(slangs['word']))):
            dic_slangs[slangs['word'][i]]=slangs['score'][i]
        # print(dic_slangs)
        for i in (range(len(GivenTweets))):
            tweet=word_tokenize(GivenTweets[i])
            for j in range(len(tweet)):
                if(tweet[j] in dic_slangs):
        # #             print(i,dic_emoji[i])
                    slangScore[i]+=dic_slangs[tweet[j]]
        return(slangScore)
    
    def vader(self, GivenTweets):
        obj= SentimentIntensityAnalyzer()
        Comp_vader=np.zeros(len(GivenTweets))
        for i in range(len(GivenTweets)):
            sentiment_dict = obj.polarity_scores(GivenTweets[i])
            Comp_vader[i]=sentiment_dict['compound']
        return(Comp_vader)
        
    def reading_ease(self,GivenTweets):
        Reading_Ease=np.zeros(len(GivenTweets))
        Reading_Grade=np.zeros(len(GivenTweets))
        for i in range(len(GivenTweets)):
            Reading_Ease[i]=flesch_reading_ease(GivenTweets[i])
            Reading_Grade[i]=flesch_kincaid_grade(GivenTweets[i])
        return(Reading_Ease,Reading_Grade)

    def POS_feature(self,flag="Test"):
        if(flag=="Test"):
            Tag_feature=self.vectorizer_pos.transform(self.TestTags)
        else:
            Tag_feature=self.vectorizer_pos.transform(self.TrainTags)
            
        return(Tag_feature)
    

In [38]:
class Data:
    def read(self, subtask="a", ids=None):
        df_tr = pd.DataFrame(pd.read_csv("olid-training-v1.0.tsv", sep="\t"))
        
        if subtask == "b":
            df_tr = df_tr[(df_tr["subtask_b"] == "UNT") | (df_tr["subtask_b"] == "TIN")]
            df_tr.reset_index(inplace=True)
        
        df_tr['tweet_original'] = df_tr['tweet']
        self.preprocess_text_caller(df_tr['tweet'])
        
        df_te = pd.DataFrame(pd.read_csv(f"testset-level{subtask}.tsv",sep="\t"))
        
        df_te['tweet_original'] = df_te['tweet']
        self.preprocess_text_caller(df_te['tweet'])
        
        print("Preprocessing done")
        
        f = Features(df_tr, df_te)
        
        print("Feature class initialised")
        
        # if you want  ngram and other features
        X_tr = self.feature_extraction(df_tr, f)
        ngram_ft = f.n_grams(df_tr["tweet"])
        pos_ft= f.POS_feature(flag="Train")
        print(X_tr.shape, ngram_ft.shape, pos_ft.shape)
        X_tr = np.column_stack((X_tr, ngram_ft.toarray(),pos_ft.toarray()))

        # if you want ngrams only
        # X_tr = f.n_grams(df_tr["tweet"])
        y_tr = df_tr[f"subtask_{subtask}"]
        
        y_te = pd.DataFrame(pd.read_csv(f"labels-level{subtask}.csv", header=None))
          
        if ids is not None:
            df_te = df_te[df_te.id.isin(ids)]
            df_te.reset_index(inplace=True)
            y_te = y_te[y_te[0].isin(ids)]
            
        # if you want all features
        X_te = self.feature_extraction(df_te, f)
        ngram_ft = f.n_grams(df_te["tweet"])
        pos_ft= f.POS_feature(flag="Test")
        
        X_te = np.column_stack((X_te, ngram_ft.toarray(), pos_ft.toarray()))
        
        #if you only want ngrams
#         X_te = f.n_grams(df_te["tweet"])

        self.df_tr = df_tr
        self.df_te = df_te
        
        return X_tr, X_te, y_tr, y_te[1]
    
    def preprocess_text(self, tweet, flag_stemm=True, flag_lemm=False, stop_words_list=None):

        ## clean (convert to lowercase and remove punctuations and characters and then strip)
        tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
        ## Tokenize (convert from string to list) and remove the stop words
    #     print(tweet)

        tokenize_tweet = tweet.split()

        if stop_words_list is not None:
            tokenize_tweet = [word for word in tokenize_tweet if word not in stop_words_list]

        ## Stemming (remove -ing, -ly, ...)
        if flag_stemm == True:
            ps = nltk.stem.porter.PorterStemmer()
            tokenize_tweet = [ps.stem(word) for word in tokenize_tweet]

        ## Lemmatisation (convert the word into root word)
        if flag_lemm == True:
            lem = nltk.stem.wordnet.WordNetLemmatizer()
            tokenize_tweet = [lem.lemmatize(word) for word in tokenize_tweet]
    #     print(tokenize_tweet)
        ## back to string from list
        
        tweet = " ".join(tokenize_tweet)
        return tweet

    def feature_extraction(self, df, f):
        Tweets_List = df["tweet_original"]
        
        df_feature = pd.DataFrame(f.FuncAllCapsCount(Tweets_List)).astype('int64')
        df_feature.rename(columns={0:"all_caps_count"},inplace=True)

        #Counting The Punctuations
        ContPunctuationCount, LastTokenPunctuation = f.FuncPunctuationsCount(Tweets_List)
        df_feature["Cont_Punc_Count"] = pd.DataFrame(ContPunctuationCount).astype('int64')
        df_feature["Last_Token_Punc"] = pd.DataFrame(LastTokenPunctuation).astype('int64')

        #Count the hash tags
        df_feature["HashTag_Count"] = pd.DataFrame(f.FuncHashtagsCount(Tweets_List)).astype('int64')
        df_feature["Anger word count"] = pd.DataFrame(f.Func_NRC_word_emotion_lexicon(Tweets_List)).astype('int64')

        df_feature["NRC_Hashtag"] = pd.DataFrame(f.Func_NRC_Hashtag_Emotion(Tweets_List))

        df_feature["NRC_10_Expanded"] = pd.DataFrame(f.Func_NRC_10_Expanded_Emotion(Tweets_List))

        df_feature["Slang"] = pd.DataFrame(f.slang(Tweets_List))

        df_feature["Vader_Sentiment"]=pd.DataFrame(f.vader(Tweets_List))
        
        ReadingEase_list,ReadingGrade_List=f.reading_ease(Tweets_List)
        df_feature["Reading_Ease"]=pd.DataFrame(ReadingEase_list)
        df_feature["Reading_Grade"]=pd.DataFrame(ReadingGrade_List)
        

        return(df_feature)
    
    def preprocess_text_caller(self, Tweets_List):
        stop_words_list=stopwords.words('english')
        for i in (range(len(Tweets_List))):
            Tweets_List[i] = self.preprocess_text(Tweets_List[i])


# Subtask A

In [39]:
d = Data()

In [40]:
X_tr, X_te, y_tr, y_te = d.read(subtask="a")

<ipython-input-38-db927b575309>:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tweets_List[i] = self.preprocess_text(Tweets_List[i])


Preprocessing done


100%|██████████| 860/860 [00:01<00:00, 728.09it/s]


ok


  2%|▏         | 308/13240 [00:00<00:04, 3079.64it/s]

ok2
Feature class initialised


100%|██████████| 13240/13240 [00:00<00:00, 43802.43it/s]


(13240, 11) (13240, 13148) (13240, 3666)


100%|██████████| 860/860 [00:00<00:00, 30207.70it/s]


In [41]:
X_tr.shape

(13240, 16825)

In [47]:
# """ Random Forest """
# parameters = {'max_depth':( 400,500,600),'n_estimators':(400)}

# rf = RandomForestClassifier(random_state=0,class_weight="balanced",n_estimators=400)
# # grid = GridSearchCV(rf, parameters,scoring='f1_macro',cv=5,verbose=True,n_jobs=-1)
# # grid.fit(csr_matrix(X_tr),y_tr)

# rf.fit(csr_matrix(X_tr), y_tr)

# y_pred = rf.predict(csr_matrix(X_te))

In [37]:
# f1_score(y_te, y_pred, average='macro')

In [38]:
# pd.DataFrame(grid.cv_results_)
# # grid.best_params_

In [27]:
# norm l2 tfidf(1,3) 0.7353976599538654 mindf 5 only tfidf(1,3)
# all features (max_iter=4000, class_weight="balanced") 0.694
log = LogisticRegression(max_iter=4000, class_weight="balanced")
log.fit(csr_matrix(X_tr), y_tr)
y_pred = log.predict(csr_matrix(X_te))
f1_score(y_te, y_pred, average="macro")

0.6940920008354219

In [51]:
# 0.7436591034788138 mindf 10 only tfidf(1,3)

""" SVM """
# svm = make_pipeline(StandardScaler(with_mean=False), SVC())
svm = SVC(kernel="linear", class_weight={"NOT":0.35,"OFF":0.65})
svm.fit(csr_matrix(X_tr), y_tr)
y_pred = svm.predict(csr_matrix(X_te))
f1_score(y_te, y_pred, average="macro")

0.7436591034788138

In [10]:
# mindf 10 0.7084396588650625 only tfidf(1,3)
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=0.001)
nb.fit(csr_matrix(X_tr), y_tr)
y_pred = nb.predict(csr_matrix(X_te))
f1_score(y_te, y_pred, average="macro")


0.6706559746285401

In [132]:
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier(base_estimator=LogisticRegression(max_iter=2500, class_weight="balanced"),n_estimators=30,n_jobs=-1,verbose=True)
bag.fit(csr_matrix(X_tr), y_tr)
y_pred = bag.predict(csr_matrix(X_te))
f1_score(y_te, y_pred, average="macro")

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  1.9min remaining:  1.9min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.0min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


0.750271052301496

# Subtask B

In [7]:
ids_b = set()

for i in range(len(y_pred)):
    if y_pred[i] == "OFF":
        ids_b.add(d.df_te["id"][i])

NameError: name 'y_pred' is not defined

In [6]:
len(ids_b)

NameError: name 'ids_b' is not defined

In [47]:
d = Data()

In [23]:
X_tr2, X_te2, y_tr2, y_te2 = d.read(subtask="b", ids=ids_b)

NameError: name 'ids_b' is not defined

In [51]:
X_te2

(78, 1863)

<13240x13629 sparse matrix of type '<class 'numpy.float64'>'
	with 375721 stored elements in Compressed Sparse Row format>

In [22]:
""" Random Forest """

rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr2, y_tr2)
y_pred = rf.predict(X_te2)
f1_score(y_te2, y_pred, average='macro')

NameError: name 'y_tr2' is not defined

In [27]:
""" SVM """

svm = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
svm.fit(X_tr2, y_tr2)
y_pred = svm.predict(X_te2)
f1_score(y_te2, y_pred, average="macro")

0.43150684931506844

# Old Code Below, Ignore

In [41]:
df_te = pd.DataFrame(pd.read_csv(f"testset-levelb.tsv",sep="\t"))
df_te['tweet_original'] = df_te['tweet']
# self.preprocess_text_caller(df_te['tweet'])

y_te = pd.DataFrame(pd.read_csv(f"labels-levelb.csv", header=None))




In [43]:
y_te

,0,1
0,15923,TIN
1,60133,TIN
2,83681,TIN
3,65507,TIN
4,12588,UNT
...,...,...
235,22569,TIN
236,48938,TIN
237,41438,TIN
238,73439,TIN


In [44]:
if ids_b is not None:
    df_te = df_te[df_te.id.isin(ids_b)]
    df_te.reset_index(inplace=True)
    y_te = y_te[y_te[0].isin(ids_b)]

In [46]:
y_te

,0,1
2,83681,TIN
4,12588,UNT
5,34263,TIN
7,58995,TIN
9,46444,TIN
...,...,...
231,71592,TIN
234,30778,TIN
236,48938,TIN
237,41438,TIN


In [20]:
len(ids_b)

96

In [90]:
df_training=pd.DataFrame(pd.read_csv("olid-training-v1.0.tsv",sep="\t"))

df_training = df_training[(df_training["subtask_b"] == "UNT") | (df_training["subtask_b"] == "TIN")]
df_training.reset_index(inplace=True)

df_training['tweet_original'] = df_training['tweet']
preprocess_text_caller(df_training['tweet'])

df_testing=pd.DataFrame(pd.read_csv("testset-levelb.tsv",sep="\t"))
df_testing['tweet_original']=df_testing['tweet']
preprocess_text_caller(df_testing['tweet'])

<ipython-input-34-2b67d72cee89>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tweets_List[i]=preprocess_text(Tweets_List[i])


# use the below statement to extract features

In [107]:
train_feature=feature_extraction(df_training['tweet_original'])
test_feature=feature_extraction(df_testing['tweet_original'])

100%|██████████| 860/860 [00:00<00:00, 14081.93it/s]


In [109]:
# ngram_range parameter (1,2) means that unigram and bigram will be taken
# Count Vecotrizer automatically preprocess the tweets

vectorizer = CountVectorizer(ngram_range=(1,4), min_df=5)
Alltweets = df_training.tweet
Alltweets = Alltweets.append(df_testing.tweet)
# fit was used to learn the vocabulary
TempVector = vectorizer.fit(Alltweets)
# Getting the vocabulary for ngrams
Vocab_ngrams = TempVector.get_feature_names()
len(Vocab_ngrams)
Vocab={}
for i in range(len(Vocab_ngrams)):
    Vocab[i]=Vocab_ngrams[i]
# Now we can use this vocabulary to individually create ngrams for train and test set
# fit transform is used here as we want the counts too
vectorizer2 = CountVectorizer(ngram_range=(1,4), vocabulary=Vocab_ngrams)
Train_ngrams=vectorizer2.fit_transform(df_training.tweet)
Test_ngrams=vectorizer2.fit_transform(df_testing.tweet)

In [113]:
# from scipy.sparse import csr_matrix, hstack
# csr_TrainFeature_without_ngram=csr_matrix(train_feature)
# csr_TestFeature_without_ngram=csr_matrix(test_feature)
# TrainFeatureFinal=hstack([Train_ngrams,csr_TrainFeature_without_ngram])
# TestFeatureFinal=hstack([Test_ngrams,csr_TestFeature_without_ngram])

In [114]:
X_tr = TrainFeatureFinal
X_te = TestFeatureFinal

In [115]:
y_tr = df_training["subtask_a"]

In [116]:
clf = RandomForestClassifier(random_state=0)
clf.fit(X_tr, y_tr)
y_pred = clf.predict(X_te)
f1_score(y_te, y_pred, average='macro')

0.6671139554087532

In [118]:
X_te2 = []
y_te2 = []

for i in range(len(y_pred)):
    if y_pred[i] == "OFF":
        X_te2.append()

(860,)

In [122]:
df_training

,id,tweet,subtask_a,subtask_b,subtask_c,tweet_original
0,86426,user she should ask a few native americans wha...,OFF,UNT,NaN,@USER She should ask a few native Americans wh...
1,90194,user user go home youre drunk user maga trump2...,OFF,TIN,IND,@USER @USER Go home you’re drunk!!! @USER #MAG...
2,16820,amazon is investigating chinese employees who ...,NOT,NaN,NaN,Amazon is investigating Chinese employees who ...
3,62688,user someone shouldvetaken this piece of shit ...,OFF,UNT,NaN,"@USER Someone should'veTaken"" this piece of sh..."
4,43605,user user obama wanted liberals amp illegals t...,NOT,NaN,NaN,@USER @USER Obama wanted liberals &amp; illega...
...,...,...,...,...,...,...
13235,95338,user sometimes i get strong vibes from people ...,OFF,TIN,IND,@USER Sometimes I get strong vibes from people...
13236,67210,benidorm creamfields maga not too shabby of a ...,NOT,NaN,NaN,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...
13237,82921,user and why report this garbage we dont give ...,OFF,TIN,OTH,@USER And why report this garbage. We don't g...
13238,27429,user pussy,OFF,UNT,NaN,@USER Pussy


In [33]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(max_iter=300)
clf.fit(TrainFeatureFinal,df_training["subtask_a"])
y_pred = clf.predict(TestFeatureFinal)
f1_score(y_te[1],y_pred,average='macro')

0.690516889390238

In [121]:
X_te[i]

TypeError: 'coo_matrix' object is not subscriptable

In [36]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier()
clf.fit(TrainFeatureFinal,df_training["subtask_a"])
y_pred=clf.predict(TestFeatureFinal)
f1_score(y_actual[1],y_pred,average='macro')

0.6913804213135069

0.6948406784472357

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf.fit(TrainFeatureFinal,df_training["subtask_a"])
y_pred=clf.predict(TestFeatureFinal)
f1_score(y_actual[1],y_pred,average='macro')

0.6669336595928211

In [129]:
for i in clf.feature_importances_:
    print(i)

8.239093224400076e-05
3.327909333922288e-05
0.0001907199955899427
1.4208336501502405e-05
1.1678946731301154e-07
6.2088682870303625e-06
2.936291530238375e-05
3.230893022150106e-05
2.044505354054651e-05
9.186229767972647e-05
1.0533889824084559e-05
4.225916204905403e-09
0.0
2.8199138706480782e-05
7.963470233051418e-05
3.2573558998978306e-05
5.358618271135379e-05
2.702253674740102e-05
3.40799729761947e-05
4.8511826242797044e-05
0.00016853144228922207
1.269466064550689e-05
5.6570292606923665e-05
2.097286590266493e-05
1.0447484314834836e-05
9.974003520611526e-06
2.3986996655335458e-05
7.025083984168037e-05
3.998008605926304e-05
0.00021917704968691857
1.547091075257025e-05
1.9895963018545803e-05
9.987332466275367e-06
2.597260716521119e-05
3.3172869958185805e-05
4.141191545043753e-06
1.2668866305055311e-05
3.002555563256503e-05
0.00011511424963207822
7.436023430893384e-06
1.1952666877980399e-05
6.493578138329404e-06
8.55103883755588e-05
3.0140394595837162e-05
6.536687232739123e-05
2.6773207424

5.925344989278565e-06
0.00016343028263427907
0.00018009792008244694
3.6546466904048177e-06
9.376452731035684e-06
1.9375238642214286e-06
3.453285666425437e-05
6.140249830456366e-05
8.324242271484609e-06
7.430791256906207e-09
1.3556973446917895e-05
4.639470477520197e-05
7.361706594576347e-05
3.432061341727827e-07
0.0
5.6819002560949715e-08
1.1590130961850215e-05
6.36878423264187e-05
0.001698281260212551
3.1193613246012427e-05
2.537982877337351e-05
0.00036031843456808875
7.127868706736029e-05
8.246378827305855e-05
0.0016616566423952731
0.0
6.252716756764352e-07
7.92795193276773e-05
2.7858599880493007e-06
5.400673209571953e-05
2.05230811698949e-06
1.745014948667866e-05
2.7840100068173885e-06
2.7328249435526406e-05
2.2647922051728133e-05
1.6909048648593184e-05
5.309259294063136e-05
1.8437464687938346e-05
3.189067303647663e-06
1.612199412877889e-05
0.00019797179459850464
0.0001111831210518755
6.900666166028626e-06
3.6714597808343986e-07
1.1034312346936987e-05
1.780199250866942e-05
0.00020386

2.9653192445442247e-05
1.983633523825086e-07
3.127974273735595e-06
5.616260238238567e-06
8.157869494706215e-06
1.0094853216584958e-05
4.511920039423478e-05
3.505308363529539e-05
0.00010763986745913879
9.43359418054669e-06
0.00019431390331772015
7.238876187086716e-05
5.1243054059207235e-05
5.2417971699706886e-05
3.731419356518564e-05
1.6906263122637977e-05
0.0003913112975187597
6.743523652808837e-05
1.6966569328444987e-05
8.388543407567274e-06
0.00010036804075311838
4.796116338432059e-06
2.991049480896481e-05
2.113110589736293e-05
2.2722515710010832e-05
2.4717016196126492e-05
2.5315303294488724e-05
6.738774204171984e-05
4.026189571325024e-06
2.4508795947540273e-07
5.620337186783128e-06
5.478008207488384e-06
1.343355268392141e-05
2.203546368891282e-05
0.0007475206765622932
1.4820052170573543e-05
4.0740925261490526e-05
6.329116525985752e-08
1.8192912232156307e-05
3.081271099892167e-05
2.5783542513672073e-06
5.824060304193081e-06
7.637737978030293e-06
2.3590094874618618e-05
1.2852540932916

7.119758207526233e-05
3.313819365299093e-05
0.00016685417967271215
5.243433169540847e-05
6.758745536864763e-05
6.829772342646306e-05
0.0001759051128346608
3.6400623950312307e-06
3.532011100336232e-07
7.19285966142767e-06
7.006758081444169e-05
1.3449717385576124e-05
3.294037023058604e-07
6.797797506361324e-05
4.6772591247821934e-05
1.7450465545239322e-05
5.09062786682859e-05
2.405613309069178e-05
0.00023560890951923494
4.589280882501291e-06
5.9749306234134534e-05
7.019522241237248e-06
1.6147046801530235e-05
1.2654137488254603e-05
2.7196829608296925e-06
1.7309741482589812e-05
2.3489558582418013e-05
1.2042495367325938e-05
0.00010367856238882296
0.0001308813125353905
8.066852212143164e-05
1.226774465707987e-05
2.21627344671003e-05
8.385333214483714e-06
0.00010052782260279352
4.491945293147631e-05
3.570614087670682e-05
1.2443655559783835e-05
6.613567883619627e-09
6.020445053790083e-06
3.367216292572812e-05
5.160322587510168e-05
7.466138115574017e-05
0.00021652053657024981
5.075099017888426e

4.300310621188869e-05
2.583796265088876e-06
2.64546767295543e-05
5.878397093911465e-05
4.872991117564665e-05
2.7058453827944396e-05
1.8668144937846894e-06
8.168788946072093e-06
0.000122137761821036
6.637241480546937e-05
2.192558484339336e-06
0.00012301973911234732
0.0003016195223547523
0.0003846965193488798
3.484238466931466e-05
1.0795480074600996e-05
0.00014319804809038132
3.350170916636605e-05
1.105553357228299e-05
6.157408501021185e-05
3.021795696346477e-05
6.935890133096432e-06
2.039565417041516e-05
7.651703684449773e-05
2.6538835693821755e-06
0.0
9.551036785774777e-05
1.3415726213431007e-05
0.00014282016749670904
5.6763043916979305e-05
1.1353920856051978e-06
4.604549364280196e-06
1.2330285874241147e-05
8.426200963356851e-05
4.45983068192677e-05
2.4128273726842672e-05
1.0192664981158378e-05
9.140447825315338e-05
4.54802252389045e-06
4.259594403187714e-06
1.9859984113024816e-05
2.4310638574303863e-06
1.7832823428870724e-05
5.352246561216925e-05
1.917269895746599e-05
8.64249887937542

In [155]:
a=func_emoji_feature(df_training.tweet_original)

[{'location': 59, 'emoji': '👊'}, {'location': 60, 'emoji': '🇺'}, {'location': 61, 'emoji': '🇸'}, {'location': 62, 'emoji': '👊'}]
OFF
[{'location': 63, 'emoji': '😂'}]
OFF
[{'location': 13, 'emoji': '🎶'}, {'location': 46, 'emoji': '♥'}, {'location': 83, 'emoji': '✔'}, {'location': 120, 'emoji': '🙉'}]
NOT
[{'location': 110, 'emoji': '😂'}]
NOT
[{'location': 111, 'emoji': '🤔'}]
OFF
[{'location': 63, 'emoji': '🤷'}, {'location': 65, 'emoji': '♂'}, {'location': 67, 'emoji': '👀'}]
NOT
[{'location': 23, 'emoji': '🗽'}, {'location': 275, 'emoji': '😀'}, {'location': 277, 'emoji': '🗽'}, {'location': 278, 'emoji': '🗽'}, {'location': 279, 'emoji': '🗽'}, {'location': 281, 'emoji': '😀'}]
NOT
[{'location': 58, 'emoji': '🙄'}]
NOT
[{'location': 32, 'emoji': '👌'}, {'location': 186, 'emoji': '💚'}, {'location': 218, 'emoji': '🤠'}, {'location': 219, 'emoji': '👍'}]
NOT
[{'location': 149, 'emoji': '❤'}]
NOT
[{'location': 54, 'emoji': '😂'}]
NOT
[{'location': 50, 'emoji': '😌'}]
NOT
[{'location': 21, 'emoji': '😢'}]

[{'location': 152, 'emoji': '😊'}]
NOT
[{'location': 68, 'emoji': '😂'}, {'location': 69, 'emoji': '😂'}, {'location': 70, 'emoji': '😂'}, {'location': 71, 'emoji': '😂'}, {'location': 72, 'emoji': '😂'}]
NOT
[{'location': 134, 'emoji': '🇺'}, {'location': 135, 'emoji': '🇸'}, {'location': 136, 'emoji': '💵'}, {'location': 137, 'emoji': '😏'}, {'location': 138, 'emoji': '🤣'}]
OFF
[{'location': 39, 'emoji': '👍'}]
NOT
[{'location': 138, 'emoji': '🤷'}, {'location': 140, 'emoji': '♀'}]
NOT
[{'location': 31, 'emoji': '🔥'}, {'location': 32, 'emoji': '🔥'}, {'location': 33, 'emoji': '🔥'}, {'location': 34, 'emoji': '🔥'}, {'location': 35, 'emoji': '❤'}, {'location': 36, 'emoji': '❤'}, {'location': 37, 'emoji': '❤'}, {'location': 38, 'emoji': '❤'}]
NOT
[{'location': 74, 'emoji': '😿'}]
OFF
[{'location': 17, 'emoji': '🍎'}, {'location': 18, 'emoji': '❤'}, {'location': 19, 'emoji': '🍏'}, {'location': 28, 'emoji': '💯'}, {'location': 29, 'emoji': '🇺'}, {'location': 30, 'emoji': '🇸'}, {'location': 31, 'emoji': '🇺

[{'location': 119, 'emoji': '😍'}, {'location': 120, 'emoji': '😩'}, {'location': 121, 'emoji': '❤'}]
NOT
[{'location': 152, 'emoji': '😲'}]
NOT
[{'location': 31, 'emoji': '💃'}, {'location': 32, 'emoji': '💃'}, {'location': 33, 'emoji': '💃'}, {'location': 34, 'emoji': '💃'}, {'location': 35, 'emoji': '💃'}, {'location': 36, 'emoji': '💃'}, {'location': 37, 'emoji': '💃'}, {'location': 38, 'emoji': '💃'}, {'location': 39, 'emoji': '💃'}, {'location': 40, 'emoji': '💃'}, {'location': 41, 'emoji': '😂'}, {'location': 42, 'emoji': '😂'}, {'location': 43, 'emoji': '😂'}, {'location': 44, 'emoji': '😂'}, {'location': 45, 'emoji': '😂'}]
NOT
[{'location': 143, 'emoji': '💙'}]
NOT
[{'location': 52, 'emoji': '🇺'}, {'location': 53, 'emoji': '🇸'}]
NOT
[{'location': 26, 'emoji': '🤑'}]
NOT
[{'location': 191, 'emoji': '🤭'}, {'location': 192, 'emoji': '🤔'}, {'location': 193, 'emoji': '😂'}]
OFF
[{'location': 76, 'emoji': '☺'}, {'location': 77, 'emoji': '🤗'}, {'location': 158, 'emoji': '☺'}, {'location': 159, 'emoji': 

In [156]:
a

{'©',
 '‼',
 '⁉',
 '▶',
 '◀',
 '☀',
 '☕',
 '☝',
 '☠',
 '☮',
 '☹',
 '☺',
 '♀',
 '♂',
 '♈',
 '♊',
 '♎',
 '♏',
 '♑',
 '♠',
 '♥',
 '♨',
 '⚓',
 '⚖',
 '⚪',
 '⚽',
 '⛏',
 '✅',
 '✈',
 '✊',
 '✌',
 '✍',
 '✔',
 '✝',
 '✨',
 '❄',
 '❌',
 '❓',
 '❗',
 '❣',
 '❤',
 '➖',
 '➡',
 '⤵',
 '⬇',
 '⭐',
 '🆗',
 '🇦',
 '🇧',
 '🇨',
 '🇪',
 '🇬',
 '🇮',
 '🇯',
 '🇱',
 '🇳',
 '🇵',
 '🇷',
 '🇸',
 '🇹',
 '🇺',
 '🇻',
 '🇾',
 '🌅',
 '🌈',
 '🌊',
 '🌌',
 '🌍',
 '🌎',
 '🌏',
 '🌙',
 '🌚',
 '🌝',
 '🌞',
 '🌟',
 '🌪',
 '🌷',
 '🌸',
 '🌹',
 '🌺',
 '🌻',
 '🌽',
 '🍀',
 '🍁',
 '🍄',
 '🍆',
 '🍎',
 '🍏',
 '🍒',
 '🍔',
 '🍕',
 '🍹',
 '🍻',
 '🍽',
 '🍿',
 '🎁',
 '🎂',
 '🎈',
 '🎉',
 '🎥',
 '🎱',
 '🎵',
 '🎶',
 '🎷',
 '🎸',
 '🏈',
 '🏘',
 '🏳',
 '🏴',
 '🏻',
 '🏼',
 '🏽',
 '🏾',
 '🏿',
 '🐇',
 '🐈',
 '🐉',
 '🐊',
 '🐍',
 '🐐',
 '🐑',
 '🐒',
 '🐕',
 '🐖',
 '🐦',
 '🐨',
 '🐩',
 '🐯',
 '🐱',
 '🐲',
 '🐶',
 '🐷',
 '🐸',
 '🐻',
 '🐾',
 '👀',
 '👂',
 '👅',
 '👆',
 '👇',
 '👈',
 '👉',
 '👊',
 '👋',
 '👌',
 '👍',
 '👎',
 '👏',
 '👐',
 '👑',
 '👔',
 '👢',
 '👤',
 '👩',
 '👮',
 '👯',
 '👰',
 '👵',
 '👶',
 '👸',
 '👹',
 '👼',
 '👽',
 '💀',
 '💁',
 '💃',
 '💉'

In [154]:
def func_emoji_feature(training):
    emoji_vocab=set()
    for i in range(len(training)):
        emoj_lis=emoji.emoji_lis(training[i])
        if(len(emoj_lis)>0):
#             print(training[i])
            print(emoj_lis)
            for pair in emoj_lis:
                emoji_vocab.add(pair['emoji'])
            print(df_training.subtask_a[i])
    return(emoji_vocab)
    

In [136]:
emoji.emoji_lis("👊🇸 andjnasld")

[{'location': 0, 'emoji': '👊'}, {'location': 1, 'emoji': '🇸'}]

In [127]:
emoji.get_emoji_regexp()

re.compile(r'(👨\u200d❤️\u200d💋\u200d👨|👩\u200d❤️\u200d💋\u200d👨|👩\u200d❤️\u200d💋\u200d👩|🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f|🏴\U000e0067\U000e0062\U000e0073\U000e0063\U000e0074\U000e007f|🏴\U000e0067\U000e0062\U000e0077\U000e006c\U000e0073\U000e007f|👨\u200d👨\u200d👦\u200d👦|👨\u200d👨\u200d👧\u200d👦|👨\u200d👨\u200d👧\u200d👧|👨\u200d👩\u200d👦\u200d👦|👨\u200d👩\u200d👧\u200d👦|👨\u200d👩\u200d👧\u200d👧|👩\u200d👩\u200d👦\u200d👦|👩\u200d👩\u200d👧\u200d👦|👩\u200d👩\u200d👧\u200d👧|👨\u200d❤\u200d💋\u200d👨|👩\u200d❤\u200d💋\u200d👨|👩\u200d❤\u200d💋\u200d👩|👨🏿\u200d🤝\u200d👨🏻|👨🏿\u200d🤝\u200d👨🏾|👨🏿\u200d🤝\u200d👨🏼|👨🏿\u200d🤝\u200d👨🏽|👨🏾\u200d🤝\u200d👨🏻|👨🏾\u200d🤝\u200d👨🏼|👨🏾\u200d🤝\u200d👨🏽|👨🏼\u200d🤝\u200d👨🏻|👨🏽\u200d🤝\u200d👨🏻|👨🏽\u200d🤝\u200d👨🏼|🧑🏿\u200d🤝\u200d🧑🏻|🧑🏿\u200d🤝\u200d🧑🏾|🧑🏿\u200d🤝\u200d🧑🏼|🧑🏿\u200d🤝\u200d🧑🏽|🧑🏿\u200d🤝\u200d🧑🏿|🧑🏻\u200d🤝\u200d🧑🏻|🧑🏾\u200d🤝\u200d🧑🏻|🧑🏾\u200d🤝\u200d🧑🏼|🧑🏾\u200d🤝\u200d🧑🏽|🧑🏾\u200d🤝\u200d🧑🏾|🧑🏼\u200d🤝\u200d🧑🏻|🧑🏼\u200d🤝\u200d🧑🏼|🧑🏽\u200d🤝\u200d🧑🏻|🧑🏽\u200d🤝\u200d🧑🏼|🧑🏽\u200d🤝\u200d🧑🏽|👩🏿